In [1]:
import tensorflow as tf
import numpy as np

model = tf.keras.models.load_model('../models/VGG16bn.h5', compile=False)
model = model.layers[-1]

In [9]:
netin = tf.keras.Input((224,224,3), dtype='uint8')
x = tf.cast(netin, 'float32')
x = tf.keras.applications.mobilenet_v2.preprocess_input(x) # !!!
#x = tf.nn.bias_add(x, [-123.68, -116.779, -103.939])
#x = tf.keras.applications.imagenet_utils.preprocess_input(x, mode='torch')
x = pruned_model(x)
model_lr = tf.keras.Model(inputs=netin, outputs=x)

In [3]:
def bn_sparsity_train(layer):
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        config = layer.get_config()
        config['gamma_regularizer'] = tf.keras.regularizers.l1(1e-3)
        newlayer = layer.__class__.from_config(config)
        return newlayer
    elif isinstance(layer, tf.keras.layers.Conv2D):
        newlayer = layer.__class__.from_config(layer.get_config())
        newlayer.kernel_regularizer = tf.keras.regularizers.l2(1e-4)
        return newlayer
    return layer.__class__.from_config(layer.get_config())

In [4]:
import imagenet
train_ds = imagenet.load_ds([224, 224])

In [5]:
train_ds = train_ds.map(lambda x,y: (tf.keras.applications.imagenet_utils.preprocess_input(x, mode='torch'),
                                     y), num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_ds = train_ds.cache()
train_ds = train_ds.shuffle(5000).repeat()

In [9]:
prune_model.compile(loss=tf.keras.losses.sparse_categorical_crossentropy, 
                    metrics=['accuracy'],
                    optimizer=tf.keras.optimizers.SGD(1e-3, momentum=0.9, nesterov=True))

In [10]:
b_sz = 64
ds = train_ds.batch(b_sz)
prune_model.fit(ds, epochs=5, initial_epoch=0, steps_per_epoch=50000//b_sz) 

Epoch 5/5
3125/3125 [==============================] - 228s 73ms/step - loss: 27.0356 - accuracy: 0.9963


In [19]:
pruned_model = apply_pruning(prune_model, 0.2)
pruned_model.summary()

Model: "vgg16_bn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 17)      476       
_________________________________________________________________
block1_bn1 (BatchNormalizati (None, 224, 224, 17)      68        
_________________________________________________________________
block1_relu1 (Activation)    (None, 224, 224, 17)      0         
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 46)      7084      
_________________________________________________________________
block1_bn2 (BatchNormalizati (None, 224, 224, 46)      184       
_________________________________________________________________
block1_relu2 (Activation)    (None, 224, 224, 46)      0  

In [7]:
inp = tf.keras.Input((224,224,3))
x = tf.cast(inp, 'float32')
#x = tf.keras.applications.mobilenet_v2.preprocess_input(x)
x = tf.keras.applications.imagenet_utils.preprocess_input(x, mode='torch')
x = pruned_model(x)
pruned_model = tf.keras.Model(inputs=inp, outputs=x)

pruned_model.save('vgg16_pruned20')